In [1]:
import pandas as pd
import numpy as np
import os
from scipy.misc import imread
import matplotlib.pyplot as plt
import random
import warnings
from keras.datasets import reuters
from keras.preprocessing import sequence
from keras import utils


Using TensorFlow backend.


In [26]:
'''Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

# path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
# text = open(path).read().lower()
ourdata = np.load('data/ourdata.npy')
text = ' '.join(ourdata[0])
text = text[:200000]
text = text.lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.fit(X, y, batch_size=128, epochs=5)



corpus length: 200000
total chars: 68
nb sequences: 66654
Vectorization...
Build model...
Epoch 1/5
66654/66654 [==============================] - 73s - loss: 2.3314 - acc: 0.3392    
Epoch 2/5
66654/66654 [==============================] - 68s - loss: 1.9180 - acc: 0.4388    
Epoch 3/5
66654/66654 [==============================] - 69s - loss: 1.7715 - acc: 0.4798    
Epoch 4/5
66654/66654 [==============================] - 69s - loss: 1.6713 - acc: 0.5055    
Epoch 5/5
66654/66654 [==============================] - 67s - loss: 1.5956 - acc: 0.5252    


In [27]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

start_index = random.randint(0, len(text) - maxlen - 1)

for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)
        

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(200):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


----- diversity: 0.2
----- Generating with seed: "ers will choose to take. <br /><br />i a"
ers will choose to take. <br /><br />i all the story of the screen the story of the film is a story is a story of the film is a story is a good the story of the film is a story is a good story is a to the film is a good should have a most of the film is a some and he was the commance the story of the film is a story of the film is a good film is a good film is a story to the story and the movie it all the story of the film is some and most and he was a not the movie it and some and some and some and some and all the story of the story of the film is a story is the most the story and some and have the one and some and some and the production to he has a the count the most and consurted and he was a really like the story of the film is a good should have a really the story and some and all the story of the film is a story is a good protention to the story and but the film is a not the film is a c